In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
!pip install statsmodels

In [3]:
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: '%.5f' % x)


In [4]:

Control = pd.read_csv("/kaggle/input/facebook-ab-test-of-bidding-feature/ab_test/control_group.csv")
Test = pd.read_csv("/kaggle/input/facebook-ab-test-of-bidding-feature/ab_test/test_group.csv" )

In [5]:
Control.head()

,Impression,Click,Purchase,Earning
0,82529,6090,665,2311
1,98050,3383,315,1743
2,82696,4168,458,1798
3,109914,4911,487,1696
4,108458,5988,441,1544


In [6]:
Test.head()

,Impression,Click,Purchase,Earning
0,120104,3217,702,1940
1,134776,3635,834,2929
2,107807,3057,423,2526
3,116445,4650,429,2281
4,145083,5201,750,2782


In [7]:
#Adım 2
Control.describe()


,Impression,Click,Purchase,Earning
count,40.00000,40.00000,40.00000,40.00000
mean,101711.45000,5100.62500,550.90000,1908.57500
std,20302.12298,1329.95777,134.11052,302.86833
min,45476.00000,2190.00000,267.00000,1254.00000
25%,85726.75000,4124.25000,470.50000,1685.75000
50%,99790.50000,5001.50000,531.50000,1975.00000
75%,115212.50000,5923.50000,638.00000,2120.00000
max,147539.00000,7959.00000,802.00000,2497.00000


In [8]:
Test.describe()

,Impression,Click,Purchase,Earning
count,40.00000,40.00000,40.00000,40.00000
mean,120512.42500,3967.55000,582.05000,2514.92500
std,18807.46662,923.07177,161.17516,282.70791
min,79034.00000,1837.00000,312.00000,1940.00000
25%,112692.25000,3376.50000,444.75000,2280.25000
50%,119291.50000,3931.50000,551.00000,2544.50000
75%,132050.75000,4660.25000,699.75000,2761.75000
max,158606.00000,6020.00000,890.00000,3171.00000


In [9]:
Control["AB"] = 'Control'
Test["AB"] = 'Test'
df = pd.concat([Control, Test])

In [10]:
df.head()

,Impression,Click,Purchase,Earning,AB
0,82529,6090,665,2311,Control
1,98050,3383,315,1743,Control
2,82696,4168,458,1798,Control
3,109914,4911,487,1696,Control
4,108458,5988,441,1544,Control


In [11]:
df.groupby("AB")["Purchase"].mean()

AB
Control   550.90000
Test      582.05000
Name: Purchase, dtype: float64

In [12]:
df.groupby("AB")["Purchase"].std()

AB
Control   134.11052
Test      161.17516
Name: Purchase, dtype: float64

In [13]:
#Hipotezlerin Kurulması
#H0: M1=M2 Kontrol ve Test grubunun satın almaları arasında anlamlı bir fark yoktur.
#H1 M1 !=M2 ......Vardır.

#Varsayım Kontrolü
#1.Normallik Varsayımı : Shapiro Testi
# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1:..sağlanmamaktadır.

test_stat, pvalue = shapiro(df.loc[df["AB"] == 'Control', 'Purchase'])
print('Test Stat %.4f , Pvalue %.4f'%(test_stat, pvalue))
#pvalue < 0.05 olmadığı için H0 reddedilemez.

test_stat, pvalue = shapiro(df.loc[df["AB"] == 'Test', 'Purchase'])
print('Test Stat %.4f , Pvalue %.4f'%(test_stat, pvalue))
#pvalue < 0.05 olmadığı için H0 reddedilemez.

Test Stat 0.9774 , Pvalue 0.5929
Test Stat 0.9589 , Pvalue 0.1541


In [14]:
#2. Varyans Homojenliği: Levene testi
# H0: Varyanslar Homojendir
# H1: Varyanslar Homojen Değildir.
test_stat, pvalue = levene(df.loc[df["AB"] == 'Control', 'Purchase'],
                                  df.loc[df["AB"] == 'Test', 'Purchase'])
print('Test Stat %.4f , Pvalue %.4f'%(test_stat, pvalue))
#Pvalue <0.05 olmadığı için H0 Reddedilemez.
#Varsayımlar sağlandığı için bağımsız iki örneklem t testi (parametrik test)
# 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
# 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)

Test Stat 2.6403 , Pvalue 0.1082


In [15]:
# HİPOTEZİN UYGULANMASI
#H0: M1=M2 Kontrol ve Test grubunun satın almaları arasında anlamlı bir fark yoktur.
#H1 ......Vardır.

test_stat, pvalue = ttest_ind(df.loc[df["AB"] == 'Control', 'Purchase'],
                              df.loc[df["AB"] == 'Test', 'Purchase'], equal_var=True)

print('Test Stat %.4f , Pvalue %.4f'%(test_stat, pvalue))

#pvalue <0.05 olmadığı için H0 reddedilemez.
#Sonuç olarak Kontrol ve test grubu arasında anlamlı bir fark yoktur.

Test Stat -0.9396 , Pvalue 0.3503


In [16]:
#GÖREV 4
#Adım 1
# Bağımsız iki örneklem t testini(parametrik) kullandım.
# Çünkü varsayımların ikisi de sağlanıyordu.
#Adım 2
# Kontrol ve test grubu arasında anlamlı bir fark görülmemektedir.
# Veriyi Earning değişkeni ile inceleyecek olursak ortalamaları arasında %30'dan fazla bir fark bulunmamaktadır.
# Belki de o şekilde incelenmelidir.

In [17]:
df.groupby("AB")["Earning"].mean()

AB
Control   1908.57500
Test      2514.92500
Name: Earning, dtype: float64

In [18]:
a =df.groupby("AB")["Earning"].mean()
a.values[0] / a.values[1]

0.7588993707565832